**Load Troop Data** 

In [9]:
import json
import csv
import pandas as pd

# Load the json file using pandas
def get_troop_data(path):
    df = pd.read_json(path)
    return df.drop(df.columns[0], axis=1)

df = get_troop_data('troopUpgradeStats.json')
df.head(5)

,name,housingSpace,village,category,subCategory,unlock,trainingTime,regenerationTimes,dps,upgrade,minLevel,seasonal,levels,resourceType
0,Barbarian,1,home,troop,troop,"{'hall': 1, 'cost': 100, 'time': 10, 'resource...",5,[],"[8, 11, 14, 18, 23, 26, 30, 34, 38, 42, 45, 48]","{'cost': [20000, 60000, 200000, 650000, 140000...",1,False,"[1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",NaN
1,Archer,1,home,troop,troop,"{'hall': 2, 'cost': 500, 'time': 60, 'resource...",6,[],"[7, 9, 12, 16, 20, 22, 25, 28, 31, 34, 37, 40]","{'cost': [30000, 80000, 300000, 800000, 200000...",1,False,"[0, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 9, 10, 11...",NaN
2,Goblin,1,home,troop,troop,"{'hall': 2, 'cost': 5000, 'time': 3600, 'resou...",7,[],"[11, 14, 19, 24, 32, 42, 52, 62, 72]","{'cost': [45000, 175000, 500000, 1200000, 2000...",1,False,"[0, 1, 2, 2, 3, 3, 4, 5, 6, 7, 7, 8, 8, 8, 9, 9]",NaN
3,Giant,5,home,troop,troop,"{'hall': 2, 'cost': 2500, 'time': 600, 'resour...",30,[],"[11, 14, 19, 24, 31, 43, 55, 62, 70, 78, 86, 94]","{'cost': [40000, 150000, 500000, 1200000, 2000...",1,False,"[0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 1...",NaN
4,Wall Breaker,2,home,troop,troop,"{'hall': 3, 'cost': 20000, 'time': 28800, 'res...",15,[],"[6, 10, 15, 20, 43, 55, 66, 75, 86, 94, 102, 110]","{'cost': [100000, 250000, 600000, 1200000, 250...",1,False,"[0, 0, 1, 2, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11...",NaN
